## Librerias

In [ ]:
library(dplyr)
library(ggplot2)
library(MASS)
library(car)
library(broom)
library(hnp)

## Data

In [ ]:
data <- read.csv("../data/processed/tabla_severidad.csv")
data$X <- NULL

In [ ]:
head(data)

In [ ]:
summary(data$suma_pagos)

## Seleccion de variables

In [ ]:
establecer_categorias_referencia <- function(df, vars_categoricas, var_exposicion = "exposicion_total") {
  df_copy <- df
  
  for (var in vars_categoricas) {
    exposicion_por_categoria <- aggregate(df_copy[[var_exposicion]], 
                                        by = list(df_copy[[var]]), 
                                        FUN = sum, na.rm = TRUE)
    names(exposicion_por_categoria) <- c("categoria", "exposicion_total")
    
    categoria_referencia <- exposicion_por_categoria$categoria[which.max(exposicion_por_categoria$exposicion_total)]
    
    cat("Variable:", var, "- Categoría de referencia:", categoria_referencia, "\n")
    
    df_copy[[var]] <- factor(df_copy[[var]])
    df_copy[[var]] <- relevel(df_copy[[var]], ref = as.character(categoria_referencia))
  }
  
  return(df_copy)
}

vars_categoricas <- c("Modelo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                     "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Grupo_Edad")

data <- establecer_categorias_referencia(data, vars_categoricas)

str(data)

## FRECUENCIA 

In [ ]:
# Modelo completo de frecuencia con offset
modelo_freq_completo <- glm(
  n_siniestros ~ valor + Modelo + Color + Carroceria + CLASE_FASECOLDA + 
                 TIPO_VEHICULO + SERVICIO + Sexo_Aseg + Grupo_Edad + 
                 offset(log(exposicion_total)),
  family = poisson(link = "log"),
  data = data
)

# Modelo nulo de frecuencia
modelo_freq_nulo <- glm(
  n_siniestros ~ offset(log(exposicion_total)),
  family = poisson(link = "log"),
  data = data
)

# Selección de variables usando stepwise
modelo_frecuencia_final <- step(
  modelo_freq_completo,
  scope = list(lower = modelo_freq_nulo, upper = modelo_freq_completo),
  direction = "both",
  trace = TRUE
)

# Resumen del modelo final
summary(modelo_frecuencia_final)

## Evaluación

In [ ]:
# Verificar sobredispersión
dispersion <- sum(residuals(modelo_frecuencia_final, type = "pearson")^2) / modelo_frecuencia_final$df.residual
cat("Parámetro de dispersión:", dispersion, "\n")

if (dispersion > 1.5) {
  cat("ADVERTENCIA: Hay evidencia de sobredispersión. Considerar modelo quasi-Poisson o binomial negativa.\n")
} else {
  cat("La dispersión parece apropiada para el modelo Poisson.\n")
}

In [ ]:
# Gráficos diagnósticos para el modelo de frecuencia
residuos_deviance <- residuals(modelo_frecuencia_final, type = "deviance")
residuos_pearson <- residuals(modelo_frecuencia_final, type = "pearson")
valores_ajustados <- fitted(modelo_frecuencia_final)

# Panel de diagnósticos (4 gráficos)
par(mfrow = c(2, 2))

# Residuos vs valores ajustados
plot(valores_ajustados, residuos_deviance,
     xlab = "Valores ajustados", 
     ylab = "Residuos deviance",
     main = "Residuos vs Ajustados - Frecuencia",
     pch = 16, col = "#4A90E2")
abline(h = 0, col = "red", lty = 2)

# QQ-plot de residuos
qqnorm(residuos_deviance, 
       main = "QQ-plot Residuos Deviance - Frecuencia",
       pch = 16, col = "#4A90E2")
qqline(residuos_deviance, col = "red")

# Scale-Location
plot(valores_ajustados, sqrt(abs(residuos_deviance)),
     xlab = "Valores ajustados",
     ylab = "√|Residuos deviance|",
     main = "Scale-Location - Frecuencia",
     pch = 16, col = "#4A90E2")

# Histograma de residuos
hist(residuos_deviance, 
     breaks = 30,
     main = "Distribución de Residuos - Frecuencia",
     xlab = "Residuos deviance",
     col = "#E8F4FD",
     border = "#4A90E2")

par(mfrow = c(1, 1))

In [ ]:
# Envelope plot para el modelo de frecuencia
envelope_frecuencia <- hnp(modelo_frecuencia_final,
                          halfnormal = TRUE,
                          plot = TRUE,
                          main = "Envelope Plot - Modelo Frecuencia",
                          col = "#4A90E2")

## Diagnóstico del ajuste del modelo

In [ ]:
# Análisis detallado del problema
cat("=== DIAGNÓSTICO DEL MODELO ===\n")
cat("Dispersión:", dispersion, "\n")
cat("Deviance/df:", deviance(modelo_frecuencia_final)/modelo_frecuencia_final$df.residual, "\n")

# Verificar distribución de la variable respuesta
cat("\n=== DISTRIBUCIÓN DE n_siniestros ===\n")
table(data$n_siniestros)

# Verificar ceros excesivos
prop_ceros <- sum(data$n_siniestros == 0) / nrow(data)
cat("\nProp. de ceros:", round(prop_ceros, 3), "\n")

# Verificar valores ajustados vs observados
mu_hat <- fitted(modelo_frecuencia_final)
cat("Rango valores ajustados:", round(range(mu_hat), 3), "\n")
cat("Media n_siniestros:", round(mean(data$n_siniestros), 3), "\n")

## Modelos alternativos

In [ ]:
# 1. Binomial Negativa (para sobredispersión)
library(MASS)

# Crear fórmula segura excluyendo variables con un solo nivel
if(exists("vars_problema") && length(vars_problema) > 0) {
  vars_validas_nb <- setdiff(c("valor", "Modelo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                               "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Grupo_Edad"), vars_problema)
} else {
  vars_validas_nb <- c("valor", "Modelo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                       "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Grupo_Edad")
}

formula_nb_str <- paste("n_siniestros ~", paste(vars_validas_nb, collapse = " + "), 
                        "+ offset(log(exposicion_total))")

cat("Fórmula Binomial Negativa:", formula_nb_str, "\n")

modelo_nb <- glm.nb(
  as.formula(formula_nb_str),
  data = data
)

# Selección stepwise para binomial negativa
modelo_nb_step <- stepAIC(modelo_nb, direction = "both", trace = FALSE)

cat("\n=== BINOMIAL NEGATIVA ===\n")
cat("Theta:", modelo_nb_step$theta, "\n")
cat("AIC:", AIC(modelo_nb_step), "\n")
summary(modelo_nb_step)

In [ ]:
# 2. Modelo Zero-Inflated Poisson (si hay exceso de ceros)
if(prop_ceros > 0.5) {
  library(pscl)
  
  # Usar fórmula segura
  formula_zip_str <- paste("n_siniestros ~", paste(vars_validas_nb, collapse = " + "), 
                           "+ offset(log(exposicion_total)) | 1")
  
  cat("Fórmula Zero-Inflated:", formula_zip_str, "\n")
  
  modelo_zip <- zeroinfl(
    as.formula(formula_zip_str),
    data = data,
    dist = "poisson"
  )
  
  cat("\n=== ZERO-INFLATED POISSON ===\n")
  cat("AIC:", AIC(modelo_zip), "\n")
  summary(modelo_zip)
} else {
  cat("\nProp. de ceros (", round(prop_ceros, 3), ") no justifica modelo zero-inflated\n")
}

In [ ]:
# 3. Diagnosticar variables con un solo nivel
cat("=== DIAGNÓSTICO DE VARIABLES CATEGÓRICAS ===\n")
vars_categoricas <- c("Color", "Carroceria", "CLASE_FASECOLDA", "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Grupo_Edad")

for(var in vars_categoricas) {
  niveles <- length(unique(data[[var]]))
  cat(sprintf("%-15s: %d niveles\n", var, niveles))
  if(niveles == 1) {
    cat("  *** PROBLEMA: Solo 1 nivel ***\n")
  }
}

# Identificar y remover variables con un solo nivel
vars_problema <- c()
for(var in vars_categoricas) {
  if(length(unique(data[[var]])) == 1) {
    vars_problema <- c(vars_problema, var)
  }
}

cat("\nVariables con un solo nivel:", ifelse(length(vars_problema) > 0, paste(vars_problema, collapse = ", "), "Ninguna"), "\n")

# 3. Transformaciones de variables predictoras (corregido)
# Crear fórmula excluyendo variables problemáticas
vars_validas <- setdiff(vars_categoricas, vars_problema)
formula_str <- paste("n_siniestros ~ log(valor) + I((2023-Modelo)^2) +", 
                     paste(vars_validas, collapse = " + "), 
                     "+ offset(log(exposicion_total))")

cat("\nFórmula a usar:", formula_str, "\n")

modelo_transf <- glm(
  as.formula(formula_str),
  family = poisson(link = "log"),
  data = data
)

# Stepwise con transformaciones
modelo_transf_step <- step(modelo_transf, direction = "both", trace = FALSE)

cat("\n=== MODELO CON TRANSFORMACIONES ===\n")
cat("AIC:", AIC(modelo_transf_step), "\n")
cat("Dispersión:", sum(residuals(modelo_transf_step, type="pearson")^2)/modelo_transf_step$df.residual, "\n")

In [ ]:
# Comparar todos los modelos
modelos_lista <- list(
  "Poisson" = modelo_frecuencia_final,
  "Quasi-Poisson" = modelo_freq_quasi,
  "Neg. Binomial" = modelo_nb_step,
  "Transformado" = modelo_transf_step
)

cat("\n=== COMPARACIÓN DE MODELOS ===\n")
for(i in 1:length(modelos_lista)) {
  modelo <- modelos_lista[[i]]
  nombre <- names(modelos_lista)[i]
  
  if(nombre != "Quasi-Poisson") {
    aic_val <- AIC(modelo)
  } else {
    aic_val <- "N/A"
  }
  
  if(nombre == "Neg. Binomial") {
    disp_val <- "N/A (theta controlled)"
  } else {
    disp_val <- round(sum(residuals(modelo, type="pearson")^2)/modelo$df.residual, 3)
  }
  
  cat(sprintf("%-15s AIC: %-8s Dispersión: %s\n", nombre, aic_val, disp_val))
}

# Seleccionar mejor modelo
mejor_modelo <- modelo_nb_step  # Cambiar según resultados
cat("\n=== MEJOR MODELO SELECCIONADO: Binomial Negativa ===\n")

In [ ]:
# Envelope para binomial negativa
envelope_nb <- hnp(mejor_modelo,
                   halfnormal = TRUE,
                   plot = TRUE,
                   main = "Envelope Plot - Binomial Negativa",
                   col = "#27AE60")

# Gráficos diagnósticos del mejor modelo
par(mfrow = c(2, 2))

res_mejor <- residuals(mejor_modelo, type = "deviance")
fit_mejor <- fitted(mejor_modelo)

plot(fit_mejor, res_mejor, main = "Residuos vs Ajustados - Mejor Modelo",
     xlab = "Ajustados", ylab = "Residuos", pch = 16, col = "#27AE60")
abline(h = 0, col = "red", lty = 2)

qqnorm(res_mejor, main = "QQ-plot - Mejor Modelo", pch = 16, col = "#27AE60")
qqline(res_mejor, col = "red")

plot(fit_mejor, sqrt(abs(res_mejor)), main = "Scale-Location - Mejor Modelo",
     xlab = "Ajustados", ylab = "√|Residuos|", pch = 16, col = "#27AE60")

hist(res_mejor, main = "Distribución Residuos - Mejor Modelo", 
     col = "#D5F4E6", border = "#27AE60", breaks = 20)

par(mfrow = c(1, 1))

## Modelo alternativo si hay sobredispersión

In [ ]:
# Si hay sobredispersión, usar quasi-Poisson
modelo_freq_quasi <- glm(
  formula(modelo_frecuencia_final),
  family = quasipoisson(link = "log"),
  data = data
)

# Comparar dispersión
cat("Dispersión Poisson:", dispersion, "\n")
cat("Dispersión quasi-Poisson:", summary(modelo_freq_quasi)$dispersion, "\n")

# Mostrar resumen del modelo quasi-Poisson
summary(modelo_freq_quasi)

### Evaluación del modelo quasi-Poisson

In [ ]:
# Gráficos diagnósticos para el modelo quasi-Poisson
residuos_quasi_deviance <- residuals(modelo_freq_quasi, type = "deviance")
residuos_quasi_pearson <- residuals(modelo_freq_quasi, type = "pearson")
valores_ajustados_quasi <- fitted(modelo_freq_quasi)

# Panel de diagnósticos (4 gráficos)
par(mfrow = c(2, 2))

# Residuos vs valores ajustados
plot(valores_ajustados_quasi, residuos_quasi_deviance,
     xlab = "Valores ajustados", 
     ylab = "Residuos deviance",
     main = "Residuos vs Ajustados - Quasi-Poisson",
     pch = 16, col = "#E74C3C")
abline(h = 0, col = "red", lty = 2)

# QQ-plot de residuos
qqnorm(residuos_quasi_deviance, 
       main = "QQ-plot - Quasi-Poisson",
       pch = 16, col = "#E74C3C")
qqline(residuos_quasi_deviance, col = "red")

# Scale-Location
plot(valores_ajustados_quasi, sqrt(abs(residuos_quasi_deviance)),
     xlab = "Valores ajustados",
     ylab = "√|Residuos deviance|",
     main = "Scale-Location - Quasi-Poisson",
     pch = 16, col = "#E74C3C")

# Histograma de residuos
hist(residuos_quasi_deviance, 
     breaks = 30,
     main = "Distribución Residuos - Quasi-Poisson",
     xlab = "Residuos deviance",
     col = "#FADBD8",
     border = "#E74C3C")

par(mfrow = c(1, 1))

In [ ]:
# Envelope plot manual para modelo quasi-Poisson
envelope_quasipoisson_manual <- function(modelo, nsim = 99, conf = 0.95) {
  n <- length(residuals(modelo))
  
  # Extraer información del modelo
  y <- modelo$y
  mu <- fitted(modelo)
  phi <- summary(modelo)$dispersion
  
  # Matriz para almacenar residuos simulados
  res_sim <- matrix(NA, n, nsim)
  
  # Simular datos y calcular residuos
  set.seed(123)
  for(i in 1:nsim) {
    # Simular respuestas desde quasi-Poisson (usando Poisson con ajuste de varianza)
    y_sim <- rpois(n, lambda = mu * phi) / phi
    
    # Ajustar modelo (con manejo de errores)
    tryCatch({
      # Crear dataframe temporal con los datos simulados
      data_temp <- modelo$data
      data_temp$y_sim <- y_sim
      
      # Ajustar modelo quasi-Poisson
      modelo_sim <- glm(y_sim ~ ., 
                       data = data_temp[, !names(data_temp) %in% c("n_siniestros")],
                       family = quasipoisson(link = "log"),
                       offset = log(data_temp$exposicion_total))
      
      res_sim[,i] <- sort(abs(residuals(modelo_sim, type = "deviance")))
    }, error = function(e) {
      res_sim[,i] <- NA
    })
  }
  
  # Calcular bandas de confianza
  alpha <- (1 - conf) / 2
  lower <- apply(res_sim, 1, quantile, probs = alpha, na.rm = TRUE)
  upper <- apply(res_sim, 1, quantile, probs = 1-alpha, na.rm = TRUE)
  
  # Crear gráfico
  res_obs <- sort(abs(residuals(modelo, type = "deviance")))
  valores_teoricos <- qnorm((1:n - 0.5) / n)
  
  plot(valores_teoricos, res_obs,
       pch = 16, col = "#E74C3C",
       xlab = "Cuantiles teóricos",
       ylab = "|Residuos deviance|",
       main = "Envelope Half-Normal - Quasi-Poisson")
  
  lines(valores_teoricos, lower, col = "red", lty = 2)
  lines(valores_teoricos, upper, col = "red", lty = 2)
  
  # Retornar información
  invisible(list(
    observed = res_obs,
    lower = lower,
    upper = upper,
    theoretical = valores_teoricos
  ))
}

# Aplicar la función envelope manual
envelope_quasi_manual <- envelope_quasipoisson_manual(modelo_freq_quasi)

In [ ]:
# Intentar usar hnp con modelo quasi-Poisson (puede funcionar o no)
tryCatch({
  envelope_quasi_hnp <- hnp(modelo_freq_quasi,
                           halfnormal = TRUE,
                           plot = TRUE,
                           main = "Envelope Plot hnp - Quasi-Poisson",
                           col = "#E74C3C")
}, error = function(e) {
  cat("Error con hnp para quasi-Poisson:", e$message, "\n")
  cat("Usando función manual desarrollada arriba.\n")
})

### Comparación de modelos

In [ ]:
# Comparación entre modelo Poisson y quasi-Poisson
cat("=== COMPARACIÓN DE MODELOS DE FRECUENCIA ===\n\n")

cat("1. MODELO POISSON:\n")
cat("   - Deviance:", deviance(modelo_frecuencia_final), "\n")
cat("   - AIC:", AIC(modelo_frecuencia_final), "\n")
cat("   - Dispersión estimada:", dispersion, "\n\n")

cat("2. MODELO QUASI-POISSON:\n")
cat("   - Deviance:", deviance(modelo_freq_quasi), "\n")
cat("   - Dispersión estimada:", summary(modelo_freq_quasi)$dispersion, "\n")
cat("   - Note: AIC no disponible para modelos quasi\n\n")

if (dispersion > 1.5) {
  cat("RECOMENDACIÓN: Usar modelo quasi-Poisson debido a la sobredispersión.\n")
} else {
  cat("RECOMENDACIÓN: El modelo Poisson es adecuado.\n")
}

# Mostrar coeficientes significativos del modelo final recomendado
modelo_final_recomendado <- if(dispersion > 1.5) modelo_freq_quasi else modelo_frecuencia_final
cat("\n=== COEFICIENTES DEL MODELO RECOMENDADO ===\n")
print(summary(modelo_final_recomendado)$coefficients[summary(modelo_final_recomendado)$coefficients[,"Pr(>|t|)"] < 0.05,])